In [101]:
import numpy
import scipy

In [102]:
class NeuralNetwork:

    # Initialise the neural network
    def __init__(self, input_nodes, hidden_nodes, output_nodes, learning_rate):
        self.input_nodes = input_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = output_nodes
        self.learning_rate = learning_rate

        # Init the weights - 0.5 that for minus values
        self.weight_input_hidden = (numpy.random.rand(self.hidden_nodes, self.input_nodes) - 0.5)
        self.weight_hidden_output = (numpy.random.rand(self.output_nodes, self.hidden_nodes) - 0.5)

        # Activation function is sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)

    # Train the network
    def train(self, input_list, target_list):

        # Convert parameter lists to 2d array
        inputs = numpy.array(input_list, ndmin=2).T
        targets = numpy.array(target_list, ndmin=2).T

        # Forward propagation
        # First layer
        hidden_inputs = numpy.dot(self.weight_input_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        # Second layer
        final_inputs = numpy.dot(self.weight_hidden_output, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        # Backpropagation
        # Error is (target - actual)
        output_errors = targets - final_outputs

        # Hidden layer error is output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.weight_hidden_output.T, output_errors)

        # Update weights
        self.weight_hidden_output -= self.learning_rate * numpy.dot((output_errors * final_outputs * (1 - final_outputs)), hidden_outputs.T)

        self.weight_input_hidden -= self.learning_rate * numpy.dot((hidden_errors * hidden_outputs * (1 - hidden_outputs)), inputs.T)

        pass

    # Query the neural network
    def query(self, input_list):

        # Convert input list to 2d array (column vector)
        inputs = numpy.array(input_list, ndmin=2).T

        # First layer
        hidden_inputs = numpy.dot(self.weight_input_hidden, inputs)
        hidden_outputs = self.activation_function(hidden_inputs)

        # Second layer
        final_inputs = numpy.dot(self.weight_hidden_output, hidden_outputs)
        final_outputs = self.activation_function(final_inputs)

        return final_outputs



    def describe(self):
        print(f'3 layer neural network with {self.input_nodes} input nodes / {self.hidden_nodes} hidden nodes / {self.output_nodes} output nodes and learning rate {self.learning_rate}')


In [103]:
n = NeuralNetwork(3,4,3,0.3)
n.describe()

3 layer neural network with 3 input nodes / 4 hidden nodes / 3 output nodes and learning rate 0.3


In [104]:
n.query([1,2,3])

array([[0.58861303],
       [0.55747329],
       [0.36496596]])